In [29]:
#pyvisa-py
import pymeasure
import pymeasure.instruments as pinstr
from pymeasure.instruments.resources import list_resources

from pymeasure.log import console_log
from pymeasure.display import Plotter
from pymeasure.experiment import Procedure, Results, Worker
from pymeasure.experiment import IntegerParameter, FloatParameter, Parameter
import pymeasure.instruments.keithley
import numpy as np
import matplotlib.pyplot as plt

In [30]:
a = list(list_resources())

0 : ASRL3::INSTR : Not known
1 : GPIB0::27::INSTR : KEITHLEY INSTRUMENTS INC.,MODEL 6517B,1355622,A12/700x 


In [31]:
a 

['ASRL3::INSTR', 'GPIB0::27::INSTR']

In [32]:
keithley = pymeasure.instruments.keithley.Keithley6517B('GPIB0::27::INSTR')

In [33]:
def measure_IV(device, vmin = 0.0, vmax = 1.0, npoints = 1000):

    for v_val in np.linspace(vmin, vmax, npoints):
        device.ramp_to_voltage(v_val)
        yield (v_val, device.current)
    device.disable_source()

In [34]:
I = []
V = []
keithley.source_voltage = 0
keithley.enable_source()
keithley.apply_voltage(voltage_range=5)
keithley.measure_current(nplc=1, current=0.000105, auto_range=True)
for (i, v) in measure_IV(keithley, npoints=100):
    I.append(i)
    V.append(V)
    print(i, v)

0.0 9.91e+37
0.010101010101010102 9.91e+37
0.020202020202020204 9.91e+37
0.030303030303030304 9.91e+37
0.04040404040404041 9.91e+37
0.05050505050505051 9.91e+37
0.06060606060606061 9.91e+37
0.07070707070707072 9.91e+37
0.08080808080808081 9.91e+37
0.09090909090909091 9.91e+37
0.10101010101010102 9.91e+37
0.11111111111111112 9.91e+37
0.12121212121212122 9.91e+37
0.13131313131313133 9.91e+37
0.14141414141414144 9.91e+37
0.15151515151515152 9.91e+37
0.16161616161616163 9.91e+37
0.17171717171717174 9.91e+37
0.18181818181818182 9.91e+37
0.19191919191919193 9.91e+37
0.20202020202020204 9.91e+37
0.21212121212121213 9.91e+37
0.22222222222222224 9.91e+37


KeyboardInterrupt: 

In [28]:
keithley.shutdown()

InvalidSession: Invalid session handle. The resource might be closed.